In [1]:
import pandas as pd

In [2]:
results = pd.read_csv('results.csv')

In [3]:
results

,id,model_receiver,model_sender,valid_json_count_r2,round2_sarc_ratio,round1_sarc_ratio,delta_sarc,flip_direction,flip
0,1,llama-3.2-3b,mistral-0.2-7b,10,1.0,1.0,0.0,none,0
1,3,llama-3.2-3b,mistral-0.2-7b,10,0.8,1.0,-0.2,none,0
2,32,llama-3.2-3b,mistral-0.2-7b,10,1.0,1.0,0.0,none,0
3,33,llama-3.2-3b,mistral-0.2-7b,10,0.5,0.9,-0.4,none,0
4,38,llama-3.2-3b,mistral-0.2-7b,10,0.3,0.9,-0.6,down,1
...,...,...,...,...,...,...,...,...,...
711284,205547,mistral-0.2-7b,qwen-2.5-7b,10,0.9,0.0,0.9,up,1
711285,205556,mistral-0.2-7b,qwen-2.5-7b,10,1.0,0.0,1.0,up,1
711286,205581,mistral-0.2-7b,qwen-2.5-7b,10,0.0,0.0,0.0,none,0
711287,205620,mistral-0.2-7b,qwen-2.5-7b,10,0.9,0.0,0.9,up,1


In [69]:
import numpy as np

In [70]:

def ci95(x):
    """Return 95% confidence interval size."""
    n = x.count()
    if n <= 1:
        return np.nan
    se = x.std(ddof=1) / np.sqrt(n)  # standard error
    return 1.96 * se

In [71]:
results['abs_delta_sarc'] = results['delta_sarc'].abs()
delta_df = results[['model_receiver', 'model_sender', 'abs_delta_sarc']]

In [72]:
aggregated_s_r = (delta_df.groupby(['model_receiver', 'model_sender'])['abs_delta_sarc'].agg(mean = 'mean', count = 'count', ci95=ci95).reset_index())




In [73]:
aggregated_s_r

,model_receiver,model_sender,mean,count,ci95
0,llama-3.2-3b,mistral-0.2-7b,0.206452,10445,0.005433
1,llama-3.2-3b,mistral-0.3-7b,0.886565,2448,0.006541
2,llama-3.2-3b,qwen-2.5-1.5b,0.048191,75796,0.000898
3,llama-3.2-3b,qwen-2.5-7b,0.347259,5294,0.010766
4,mistral-0.2-7b,llama-3.2-3b,0.891995,10445,0.003471
5,mistral-0.2-7b,mistral-0.3-7b,0.915113,13669,0.003129
6,mistral-0.2-7b,qwen-2.5-1.5b,0.371561,69725,0.003144
7,mistral-0.2-7b,qwen-2.5-7b,0.868438,12042,0.004675
8,mistral-0.3-7b,llama-3.2-3b,0.246410,2448,0.012187
9,mistral-0.3-7b,mistral-0.2-7b,0.178579,13669,0.004498


In [ ]:

delta_sender_receiver = aggregated_s_r.pivot(index='model_sender', columns='model_receiver', values='abs_delta_sarc')

In [40]:
delta_sender_receiver

model_receiver,llama-3.2-3b,mistral-0.2-7b,mistral-0.3-7b,qwen-2.5-1.5b,qwen-2.5-7b
model_sender,,,,,
llama-3.2-3b,NaN,0.891995,0.246410,0.957035,0.704998
mistral-0.2-7b,0.206452,NaN,0.178579,0.972347,0.352512
mistral-0.3-7b,0.886565,0.915113,NaN,0.974908,0.948945
qwen-2.5-1.5b,0.048191,0.371561,0.061278,NaN,0.078590
qwen-2.5-7b,0.347259,0.868438,0.204065,0.973459,NaN


In [46]:
table = delta_sender_receiver.to_latex(
    index=True,
    float_format=lambda x: f"{x:.3f}",
    caption="Abs delta sarcasm by sender/receiver",
    label="tab:abs_delta_sarc",
    bold_rows=True,
    na_rep="NaN"
)

print(table)

\begin{table}
\caption{Abs delta sarcasm by sender/receiver}
\label{tab:abs_delta_sarc}
\begin{tabular}{lrrrrr}
\toprule
model_receiver & llama-3.2-3b & mistral-0.2-7b & mistral-0.3-7b & qwen-2.5-1.5b & qwen-2.5-7b \\
model_sender &  &  &  &  &  \\
\midrule
\textbf{llama-3.2-3b} & NaN & 0.892 & 0.246 & 0.957 & 0.705 \\
\textbf{mistral-0.2-7b} & 0.206 & NaN & 0.179 & 0.972 & 0.353 \\
\textbf{mistral-0.3-7b} & 0.887 & 0.915 & NaN & 0.975 & 0.949 \\
\textbf{qwen-2.5-1.5b} & 0.048 & 0.372 & 0.061 & NaN & 0.079 \\
\textbf{qwen-2.5-7b} & 0.347 & 0.868 & 0.204 & 0.973 & NaN \\
\bottomrule
\end{tabular}
\end{table}



In [63]:
delta_s = delta_df.drop('model_receiver', axis=1)
aggregated_s = delta_s.groupby('model_sender').mean().reset_index()

In [64]:
aggregated_s

,model_sender,abs_delta_sarc
0,llama-3.2-3b,0.917111
1,mistral-0.2-7b,0.723846
2,mistral-0.3-7b,0.963689
3,qwen-2.5-1.5b,0.133075
4,qwen-2.5-7b,0.885423


In [65]:
table = aggregated_s.to_latex(
    index=False,
    float_format=lambda x: f"{x:.3f}",
    caption="Abs delta sarcasm by sender",
    label="tab:abs_delta_sarc",
    bold_rows=True,
    na_rep="NaN"
)


In [66]:
print(table)

\begin{table}
\caption{Abs delta sarcasm by sender}
\label{tab:abs_delta_sarc}
\begin{tabular}{lr}
\toprule
model_sender & abs_delta_sarc \\
\midrule
llama-3.2-3b & 0.917 \\
mistral-0.2-7b & 0.724 \\
mistral-0.3-7b & 0.964 \\
qwen-2.5-1.5b & 0.133 \\
qwen-2.5-7b & 0.885 \\
\bottomrule
\end{tabular}
\end{table}

